# Information Retrieval 1#
## Assignment 2: Retrieval models [100 points] ##

In this assignment you will get familiar with basic and advanced information retrieval concepts. You will implement different information retrieval ranking models and evaluate their performance.

We provide you with a Indri index. To query the index, you'll use a Python package ([pyndri](https://github.com/cvangysel/pyndri)) that allows easy access to the underlying document statistics.

For evaluation you'll use the [TREC Eval](https://github.com/usnistgov/trec_eval) utility, provided by the National Institute of Standards and Technology of the United States. TREC Eval is the de facto standard way to compute Information Retrieval measures and is frequently referenced in scientific papers.

This is a **groups-of-three assignment**, the deadline is **Wednesday, January 31st**. Code quality, informative comments and convincing analysis of the results will be considered when grading. Submission should be done through blackboard, questions can be asked on the course [Piazza](piazza.com/university_of_amsterdam/spring2018/52041inr6y/home).

### Technicalities (must-read!) ###

The assignment directory is organized as follows:
   * `./assignment.ipynb` (this file): the description of the assignment.
   * `./index/`: the index we prepared for you.
   * `./ap_88_90/`: directory with ground-truth and evaluation sets:
      * `qrel_test`: test query relevance collection (**test set**).
      * `qrel_validation`: validation query relevance collection (**validation set**).
      * `topics_title`: semicolon-separated file with query identifiers and terms.

You will need the following software packages (tested with Python 3.5 inside [Anaconda](https://conda.io/docs/user-guide/install/index.html)):
   * Python 3.5 and Jupyter
   * Indri + Pyndri (Follow the installation instructions [here](https://github.com/nickvosk/pyndri/blob/master/README.md))
   * gensim [link](https://radimrehurek.com/gensim/install.html)
   * TREC Eval [link](https://github.com/usnistgov/trec_eval)

### TREC Eval primer ###
The TREC Eval utility can be downloaded and compiled as follows:

    git clone https://github.com/usnistgov/trec_eval.git
    cd trec_eval
    make

TREC Eval computes evaluation scores given two files: ground-truth information regarding relevant documents, named *query relevance* or *qrel*, and a ranking of documents for a set of queries, referred to as a *run*. The *qrel* will be supplied by us and should not be changed. For every retrieval model (or combinations thereof) you will generate a run of the top-1000 documents for every query. The format of the *run* file is as follows:

    $query_identifier Q0 $document_identifier $rank_of_document_for_query $query_document_similarity $run_identifier
    
where
   * `$query_identifier` is the unique identifier corresponding to a query (usually this follows a sequential numbering).
   * `Q0` is a legacy field that you can ignore.
   * `$document_identifier` corresponds to the unique identifier of a document (e.g., APXXXXXXX where AP denotes the collection and the Xs correspond to a unique numerical identifier).
   * `$rank_of_document_for_query` denotes the rank of the document for the particular query. This field is ignored by TREC Eval and is only maintained for legacy support. The ranks are computed by TREC Eval itself using the `$query_document_similarity` field (see next). However, it remains good practice to correctly compute this field.
   * `$query_document_similarity` is a score indicating the similarity between query and document where a higher score denotes greater similarity.
   * `$run_identifier` is an identifier of the run. This field is for your own convenience and has no purpose beyond bookkeeping.
   
For example, say we have two queries: `Q1` and `Q2` and we rank three documents (`DOC1`, `DOC2`, `DOC3`). For query `Q1`, we find the following similarity scores `score(Q1, DOC1) = 1.0`, `score(Q1, DOC2) = 0.5`, `score(Q1, DOC3) = 0.75`; and for `Q2`: `score(Q2, DOC1) = -0.1`, `score(Q2, DOC2) = 1.25`, `score(Q1, DOC3) = 0.0`. We can generate run using the following snippet:

In [2]:
import logging
import sys
import os
import time
import math
import numpy as np
import subprocess
import functools
import random
from scipy import stats


def write_run(model_name, data, out_f,
              max_objects_per_query=sys.maxsize,
              skip_sorting=False):
    """
    Write a run to an output file.
    Parameters:
        - model_name: identifier of run.
        - data: dictionary mapping topic_id to object_assesments;
            object_assesments is an iterable (list or tuple) of
            (relevance, object_id) pairs.
            The object_assesments iterable is sorted by decreasing order.
        - out_f: output file stream.
        - max_objects_per_query: cut-off for number of objects per query.
    """
    for subject_id, object_assesments in data.items():
        if not object_assesments:
            logging.warning('Received empty ranking for %s; ignoring.',
                            subject_id)

            continue

        # Probe types, to make sure everything goes alright.
        # assert isinstance(object_assesments[0][0], float) or \
        #     isinstance(object_assesments[0][0], np.float32)
        assert isinstance(object_assesments[0][1], str) or \
            isinstance(object_assesments[0][1], bytes)

        if not skip_sorting:
            object_assesments = sorted(object_assesments, reverse=True)

        if max_objects_per_query < sys.maxsize:
            object_assesments = object_assesments[:max_objects_per_query]

        if isinstance(subject_id, bytes):
            subject_id = subject_id.decode('utf8')

        for rank, (relevance, object_id) in enumerate(object_assesments):
            if isinstance(object_id, bytes):
                object_id = object_id.decode('utf8')

            out_f.write(
                '{subject} Q0 {object} {rank} {relevance} '
                '{model_name}\n'.format(
                    subject=subject_id,
                    object=object_id,
                    rank=rank + 1,
                    relevance=relevance,
                    model_name=model_name))
            
# The following writes the run to standard output.
# In your code, you should write the runs to local
# storage in order to pass them to trec_eval.
write_run(
    model_name='example',
    data={
        'Q1': ((1.0, 'DOC1'), (0.5, 'DOC2'), (0.75, 'DOC3')),
        'Q2': ((-0.1, 'DOC1'), (1.25, 'DOC2'), (0.0, 'DOC3')),
    },
    out_f=sys.stdout,
    max_objects_per_query=1000)

Q1 Q0 DOC1 1 1.0 example
Q1 Q0 DOC3 2 0.75 example
Q1 Q0 DOC2 3 0.5 example
Q2 Q0 DOC2 1 1.25 example
Q2 Q0 DOC3 2 0.0 example
Q2 Q0 DOC1 3 -0.1 example


Now, imagine that we know that `DOC1` is relevant and `DOC3` is non-relevant for `Q1`. In addition, for `Q2` we only know of the relevance of `DOC3`. The query relevance file looks like:

    Q1 0 DOC1 1
    Q1 0 DOC3 0
    Q2 0 DOC3 1
    
We store the run and qrel in files `example.run` and `example.qrel` respectively on disk. We can now use TREC Eval to compute evaluation measures. In this example, we're only interested in Mean Average Precision and we'll only show this below for brevity. However, TREC Eval outputs much more information such as NDCG, recall, precision, etc.

    $ trec_eval -m all_trec -q example.qrel example.run | grep -E "^map\s"
    > map                   	Q1	1.0000
    > map                   	Q2	0.5000
    > map                   	all	0.7500
    
Now that we've discussed the output format of rankings and how you can compute evaluation measures from these rankings, we'll now proceed with an overview of the indexing framework you'll use.

### Pyndri primer ###
For this assignment you will use [Pyndri](https://github.com/cvangysel/pyndri) [[1](https://arxiv.org/abs/1701.00749)], a python interface for [Indri](https://www.lemurproject.org/indri.php). We have indexed the document collection and you can query the index using Pyndri. We will start by giving you some examples of what Pyndri can do:

First we read the document collection index with Pyndri:

In [3]:
import pyndri

index = pyndri.Index('index/')

The loaded index can be used to access a collection of documents in an easy manner. We'll give you some examples to get some idea of what it can do, it is up to you to figure out how to use it for the remainder of the assignment.

First let's look at the number of documents, since Pyndri indexes the documents using incremental identifiers we can simply take the lowest index and the maximum document and consider the difference:

In [4]:
print("There are %d documents in this collection." % (index.maximum_document() - index.document_base()))

There are 164597 documents in this collection.


Let's take the first document out of the collection and take a look at it:

In [5]:
example_document = index.document(index.document_base())
print(example_document)

('AP890425-0001', (1360, 192, 363, 0, 880, 0, 200, 0, 894, 412, 92160, 3, 192, 0, 363, 34, 1441, 0, 174134, 0, 200, 0, 894, 412, 2652, 0, 810, 107, 49, 4903, 420, 0, 1, 48, 35, 489, 0, 35, 687, 192, 243, 0, 249311, 1877, 0, 1651, 1174, 0, 2701, 117, 412, 0, 810, 391, 245233, 1225, 5838, 16, 0, 233156, 3496, 0, 393, 17, 0, 2435, 4819, 930, 0, 0, 200, 0, 894, 0, 22, 398, 145, 0, 3, 271, 115, 0, 1176, 2777, 292, 0, 725, 192, 0, 0, 50046, 0, 1901, 1130, 0, 192, 0, 408, 0, 243779, 0, 0, 553, 192, 0, 363, 0, 3747, 0, 0, 0, 0, 1176, 0, 1239, 0, 0, 1115, 17, 0, 0, 585, 192, 1963, 0, 0, 412, 54356, 0, 773, 0, 0, 0, 192, 0, 0, 1130, 0, 363, 0, 545, 192, 0, 1174, 1901, 1130, 0, 4, 398, 145, 39, 0, 577, 0, 355, 0, 491, 0, 6025, 0, 0, 193156, 88, 34, 437, 0, 0, 1852, 0, 828, 0, 1588, 0, 0, 0, 2615, 0, 0, 107, 49, 420, 0, 0, 190, 7, 714, 2701, 0, 237, 192, 157, 0, 412, 34, 437, 0, 0, 200, 6025, 26, 0, 0, 0, 0, 363, 0, 22, 398, 145, 0, 200, 638, 126222, 6018, 0, 880, 0, 0, 161, 0, 0, 319, 894, 2701, 

Here we see a document consists of two things, a string representing the external document identifier and an integer list representing the identifiers of words that make up the document. Pyndri uses integer representations for words or terms, thus a token_id is an integer that represents a word whereas the token is the actual text of the word/term. Every id has a unique token and vice versa with the exception of stop words: words so common that there are uninformative, all of these receive the zero id.

To see what some ids and their matching tokens we take a look at the dictionary of the index:

In [6]:
token2id, id2token, _ = index.get_dictionary()
print(list(id2token.items())[:15])

[(1, 'new'), (2, 'percent'), (3, 'two'), (4, '1'), (5, 'people'), (6, 'million'), (7, '000'), (8, 'government'), (9, 'president'), (10, 'years'), (11, 'state'), (12, '2'), (13, 'states'), (14, 'three'), (15, 'time')]


Using this dictionary we can see the tokens for the (non-stop) words in our example document:

In [7]:
print([id2token[word_id] for word_id in example_document[1] if word_id > 0])

['52', 'students', 'arrested', 'takeover', 'university', 'massachusetts', 'building', 'fifty', 'two', 'students', 'arrested', 'tuesday', 'evening', 'occupying', 'university', 'massachusetts', 'building', 'overnight', 'protest', 'defense', 'department', 'funded', 'research', 'new', 'york', 'city', 'thousands', 'city', 'college', 'students', 'got', 'unscheduled', 'holiday', 'demonstrators', 'occupied', 'campus', 'administration', 'building', 'protest', 'possible', 'tuition', 'increases', 'prompting', 'officials', 'suspend', 'classes', '60', 'police', 'riot', 'gear', 'arrived', 'university', 'massachusetts', '5', 'p', 'm', 'two', 'hours', 'later', 'bus', 'drove', 'away', '29', 'students', 'camped', 'memorial', 'hall', 'students', 'charged', 'trespassing', '23', 'students', 'arrested', 'lying', 'bus', 'prevent', 'leaving', 'police', '300', 'students', 'stood', 'building', 'chanting', 'looking', 'students', 'hall', 'arrested', '35', 'students', 'occupied', 'memorial', 'hall', '1', 'p', 'm',

The reverse can also be done, say we want to look for news about the "University of Massachusetts", the tokens of that query can be converted to ids using the reverse dictionary:

In [8]:
query_tokens = index.tokenize("University of Massachusetts")
print("Query by tokens:", query_tokens)
query_id_tokens = [token2id.get(query_token,0) for query_token in query_tokens]
print("Query by ids with stopwords:", query_id_tokens)
query_id_tokens = [word_id for word_id in query_id_tokens if word_id > 0]
print("Query by ids without stopwords:", query_id_tokens)

Query by tokens: ['university', '', 'massachusetts']
Query by ids with stopwords: [200, 0, 894]
Query by ids without stopwords: [200, 894]


Naturally we can now match the document and query in the id space, let's see how often a word from the query occurs in our example document:

In [9]:
matching_words = sum([True for word_id in example_document[1] if word_id in query_id_tokens])
print("Document %s has %d word matches with query: \"%s\"." % (example_document[0], matching_words, ' '.join(query_tokens)))
print("Document %s and query \"%s\" have a %.01f%% overlap." % (example_document[0], ' '.join(query_tokens),matching_words/float(len(example_document[1]))*100))

Document AP890425-0001 has 13 word matches with query: "university  massachusetts".
Document AP890425-0001 and query "university  massachusetts" have a 2.5% overlap.


While this is certainly not everything Pyndri can do, it should give you an idea of how to use it. Please take a look at the [examples](https://github.com/cvangysel/pyndri) as it will help you a lot with this assignment.

**CAUTION**: Avoid printing out the whole index in this Notebook as it will generate a lot of output and is likely to corrupt the Notebook.

### Parsing the query file
You can parse the query file (`ap_88_89/topics_title`) using the following snippet:

In [10]:
import collections
import io
import logging
import sys

def parse_topics(file_or_files,
                 max_topics=sys.maxsize, delimiter=';'):
    assert max_topics >= 0 or max_topics is None

    topics = collections.OrderedDict()

    if not isinstance(file_or_files, list) and \
            not isinstance(file_or_files, tuple):
        if hasattr(file_or_files, '__iter__'):
            file_or_files = list(file_or_files)
        else:
            file_or_files = [file_or_files]

    for f in file_or_files:
        assert isinstance(f, io.IOBase)

        for line in f:
            assert(isinstance(line, str))

            line = line.strip()

            if not line:
                continue

            topic_id, terms = line.split(delimiter, 1)

            if topic_id in topics and (topics[topic_id] != terms):
                    logging.error('Duplicate topic "%s" (%s vs. %s).',
                                  topic_id,
                                  topics[topic_id],
                                  terms)

            topics[topic_id] = terms

            if max_topics > 0 and len(topics) >= max_topics:
                break

    return topics

with open('./ap_88_89/topics_title', 'r') as f_topics:
    print(parse_topics([f_topics]))

OrderedDict([('51', 'Airbus Subsidies'), ('52', 'South African Sanctions'), ('53', 'Leveraged Buyouts'), ('54', 'Satellite Launch Contracts'), ('55', 'Insider Trading'), ('56', 'Prime (Lending) Rate Moves, Predictions'), ('57', 'MCI'), ('58', 'Rail Strikes'), ('59', 'Weather Related Fatalities'), ('60', 'Merit-Pay vs. Seniority'), ('61', 'Israeli Role in Iran-Contra Affair'), ('62', "Military Coups D'etat"), ('63', 'Machine Translation'), ('64', 'Hostage-Taking'), ('65', 'Information Retrieval Systems'), ('66', 'Natural Language Processing'), ('67', 'Politically Motivated Civil Disturbances'), ('68', 'Health Hazards from Fine-Diameter Fibers'), ('69', 'Attempts to Revive the SALT II Treaty'), ('70', 'Surrogate Motherhood'), ('71', 'Border Incursions'), ('72', 'Demographic Shifts in the U.S.'), ('73', 'Demographic Shifts across National Boundaries'), ('74', 'Conflicting Policy'), ('75', 'Automation'), ('76', 'U.S. Constitution - Original Intent'), ('77', 'Poaching'), ('78', 'Greenpeace'

### Task 1: Implement and compare lexical IR methods [35 points] ### 

In this task you will implement a number of lexical methods for IR using the **Pyndri** framework. Then you will evaluate these methods on the dataset we have provided using **TREC Eval**.

Use the **Pyndri** framework to get statistics of the documents (term frequency, document frequency, collection frequency; **you are not allowed to use the query functionality of Pyndri**) and implement the following scoring methods in **Python**:

- [TF-IDF](http://nlp.stanford.edu/IR-book/html/htmledition/tf-idf-weighting-1.html) and 
- [BM25](http://nlp.stanford.edu/IR-book/html/htmledition/okapi-bm25-a-non-binary-model-1.html) with k1=1.2 and b=0.75. **[5 points]**
- Language models ([survey](https://drive.google.com/file/d/0B-zklbckv9CHc0c3b245UW90NE0/view))
    - Jelinek-Mercer (explore different values of 𝛌 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - Dirichlet Prior (explore different values of 𝛍 [500, 1000, 1500]). **[5 points]**
    - Absolute discounting (explore different values of 𝛅 in the range [0.1, 0.5, 0.9]). **[5 points]**
    - [Positional Language Models](http://sifaka.cs.uiuc.edu/~ylv2/pub/sigir09-plm.pdf) define a language model for each position of a document, and score a document based on the scores of its PLMs. The PLM is estimated based on propagated counts of words within a document through a proximity-based density function, which both captures proximity heuristics and achieves an effect of “soft” passage retrieval. Implement the PLM, all five kernels, but only the Best position strategy to score documents. Use 𝛔 equal to 50, and Dirichlet smoothing with 𝛍 optimized on the validation set (decide how to optimize this value yourself and motivate your decision in the report). **[10 points]**
    
Implement the above methods and report evaluation measures (on the test set) using the hyper parameter values you optimized on the validation set (also report the values of the hyper parameters). Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

For the language models, create plots showing `NDCG@10` with varying values of the parameters. You can do this by chaining small scripts using shell scripting (preferred) or execute trec_eval using Python's `subprocess`.

Compute significance of the results using a [two-tailed paired Student t-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html) **[5 points]**. Be wary of false rejection of the null hypothesis caused by the [multiple comparisons problem](https://en.wikipedia.org/wiki/Multiple_comparisons_problem). There are multiple ways to mitigate this problem and it is up to you to choose one.

Analyse the results by identifying specific queries where different methods succeed or fail and discuss possible reasons that cause these differences. This is *very important* in order to understand who the different retrieval functions behave.

**NOTE**: Don’t forget to use log computations in your calculations to avoid underflows. 

**IMPORTANT**: You should structure your code around the helper functions we provide below.

In [11]:
with open('./ap_88_89/topics_title', 'r') as f_topics:
    queries = parse_topics([f_topics])

index = pyndri.Index('index/')
# the total number of documents
num_documents = index.maximum_document() - index.document_base()

# the pyindri dictionary
dictionary = pyndri.extract_dictionary(index)

# tokenize the queries
# query_id --> [token_id1, token_id2, ...] 
tokenized_queries = {
    query_id: [dictionary.translate_token(token)
               for token in index.tokenize(query_string)
               if dictionary.has_token(token)]
    for query_id, query_string in queries.items()}

# gather all of the query token_ids into a set TODO why? 
query_term_ids = set(
    query_term_id
    for query_term_ids in tokenized_queries.values()
    for query_term_id in query_term_ids)

print('Gathering statistics about', len(query_term_ids), 'terms.')

#### Inverted index creation. 
# per document dictionaries
document_lengths = {} # doc_id -> length of document
unique_terms_per_document = {} # doc_id -> number of unique tokens in document

# (query_term, (doc_id)) -> document TF
inverted_index = collections.defaultdict(dict)
# (query_term_id) -> frequency of occurence of query_term_id in all of the documents 
collection_frequencies = collections.defaultdict(int)
# total number of words in the document
total_terms = 0

start_time = time.time()
doc_id_to_ext_id = {}

for int_doc_id in range(index.document_base(), index.maximum_document()):
    ext_doc_id, doc_token_ids = index.document(int_doc_id)
    doc_id_to_ext_id[int_doc_id] = ext_doc_id
    
    document_bow = collections.Counter(
        token_id for token_id in doc_token_ids
        if token_id > 0)
    
    
    document_length = sum(document_bow.values())

    document_lengths[int_doc_id] = document_length
    total_terms += document_length

    unique_terms_per_document[int_doc_id] = len(document_bow)
    
    # accumulate TF and CF
    for query_term_id in query_term_ids:
        assert query_term_id is not None

        document_term_frequency = document_bow.get(query_term_id, 0)

        if document_term_frequency == 0:
            continue

        collection_frequencies[query_term_id] += document_term_frequency
        inverted_index[query_term_id][int_doc_id] = document_term_frequency
    
avg_doc_length = total_terms / num_documents

print('Inverted index creation took', time.time() - start_time, 'seconds.')

Gathering statistics about 456 terms.
Inverted index creation took 27.21049475669861 seconds.


In [12]:
class SparseQueryMatrix:
    def __init__(self, all_document_ids):
        self.matrix = collections.defaultdict(lambda: collections.defaultdict(float))
        self.all_document_ids = all_document_ids
    
    def set_value(self, doc_id, query_id, value):
        self.matrix[doc_id][query_id] = value
    
    def dot(self, va, vb):
        dot_product = 0.0
        common_terms = set(va.keys()).intersect(vb.keys())
        
        for term in common_terms:
            dot_product += va[term] * va[term]
        
        return dot_product
    
    def length(self, v):
        length = 0
        for key, value in v.items():
            length += math.pow(value, 2)
        return math.sqrt(length)
        
    
    def cosine_distance(self, va, vb):
        return self.dot(va, vb) / (self.length(va) * self.length(vb))
    
    def compute(self, query_vector, distance_metric = "cosine"):
        scores = []
        
        for doc_id in self.all_document_ids:
            if doc_id not in self.matrx:
                scores.append((0.0, doc_id))
                continue
            distance = 0.0
            if distance_metric == "cosine":
                distance = self.cosine_distance(self.matrix[doc_id], query_vector)
            else:
                raise ValueError()
            scores.append((distance, doc_id))
        
        return scores
        
        

In [13]:
def run_retrieval(model_name, score_fn):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    #if os.path.exists(run_out_path):
    #    return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    
    
    score_sums = collections.defaultdict(lambda: collections.defaultdict(float))
    
    for query_id, query_terms in queries.items():
        for query_term_id in tokenized_queries[query_id]:
            # TODO what happens when the query term doesn't occur in any document
            for doc_id, document_term_freq in inverted_index[query_term_id].items():
                score = score_fn(doc_id, query_term_id, document_term_freq)
                score_sums[query_id][doc_id] += score
                
    data = {}
    for query_id, document_scores in score_sums.items():
        doc_scores = []
        for doc_id in document_scores.keys():
            doc_scores.append((document_scores[doc_id], doc_id_to_ext_id[doc_id]))
        doc_scores.sort(key = lambda _: -_[0])
        data[query_id] = doc_scores
    
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

In [14]:
def tfidf(int_document_id, query_term_id, document_term_freq):
    """
    Scoring function for a document and a query term
    
    :param int_document_id: the document id
    :param query_token_id: the query term id (assuming you have split the query to tokens)
    :param document_term_freq: the document term frequency of the query term 
    """
    idf = math.log(num_documents / len(inverted_index[query_term_id]))
    tf = math.log(1 + document_term_freq)
    return tf * idf

In [15]:
def bm25(int_document_id, query_term_id, document_term_freq, k_1 = 1.2, b = 0.75):
    """
    TODO
    """
    idf = math.log(num_documents / len(inverted_index[query_term_id]))
    tf_numerator = (k_1 + 1) * document_term_freq 
    tf_denom = k_1 * ((1 - b) + b * (document_lengths[int_document_id] / avg_doc_length)) + document_term_freq
    return idf * (tf_numerator / tf_denom)

In [16]:
def jelinek_mercer(int_document_id, query_term_id, document_term_freq, lmbd = 0.5):
    """
    TODO
    """
    document_contrib = lmbd * (document_term_freq / document_lengths[int_document_id]) 
    corpus_contrib = (1 - lmbd) * (collection_frequencies[query_term_id] / total_terms)
    return np.log(document_contrib + corpus_contrib)

In [17]:
def dirichlet_prior(int_document_id, query_term_id, document_term_freq, mu = 1000):
    """
    TODO
    """
    #print("mu", mu)
    numerator = document_term_freq + mu * (collection_frequencies[query_term_id] / total_terms)
    denom = document_lengths[int_document_id] + mu
    return np.log(numerator / denom)

In [18]:
def absolute_discounting(int_document_id, query_term_id, document_term_freq, delta = 0.1):
    """
    sigma: discount constant [0,1]
    delta: sigma*|du|/|d|
    """
    sigma = (delta * unique_terms_per_document[int_document_id]) / document_lengths[int_document_id]
    numerator = max(document_term_freq - delta, 0) 
    denominator = document_lengths[int_document_id]
    discounted_prob = numerator / denominator
    unigram_prob = collection_frequencies[query_term_id] / total_terms
    return np.log(discounted_prob + (sigma * unigram_prob))

In [19]:
 #please check, by ece

def gaussian_kernel(i, j, sigma):
    return np.exp(-np.power((i-j),2)/(2*np.power(sigma,2)))

def gaussian_kernel_estimation(i, j, sigma, N):
    #TODO the one with the cumulative density function CDF
    pass
    
def triangle_kernel(i, j, sigma): 
    absolute = np.absolute(i-j)
    return 1 - (absolute/sigma) if(absolute <= sigma) else 0

def cosine_kernel(i, j, sigma):
    absolute = np.absolute(i-j) 
    k = 0.0
    
    if absolute <= sigma:
        k = (1 + np.cos((absolute*math.pi)/sigma))/2
    
    return k

def circle_kernel(i, j, sigma):
    absolute = np.absolute(i-j) 
    k = 0.0
    
    if absolute <= sigma:
        k = np.sqrt(1 - np.power(absolute/sigma, 2))
    
    return k

def passage_kernel(i, j, sigma):
    absolute = np.absolute(i-j)
    return 1 if(absolute <= sigma) else 0


In [24]:
def run_retrieval_plm(model_name, score_fn):
    
    #please check, by ece
    
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param score_fn: the scoring function (a function - see below for an example) 
    """
    run_out_path = '{}.run'.format(model_name)

    #if os.path.exists(run_out_path):
    #    return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    
    
    score_sums = collections.defaultdict(lambda: collections.defaultdict(float))
    
    plm_models_wdi = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(float)))
            
    scores_qdi = collections.defaultdict(lambda: collections.defaultdict(lambda: collections.defaultdict(float)))
                
    
    #MLE probs for query_lm[query_id][query_term_id]
    query_lm = collections.defaultdict(lambda: collections.defaultdict(float)) 
    
    for query_id, query_terms in queries.items():
        query_word_counts = collections.defaultdict(float)
        tokenized_qs = tokenized_queries[query_id]
        for query_term_id in tokenized_qs:
            query_word_counts[query_term_id] += 1 #P(w_i|q)
         
        query_word_counts.update((k,k/len(tokenized_qs)) for k in query_word_counts)
        
        query_lm[query_id] = query_word_counts #for each query, keep LM = P(w|q) for each unique word in the query
       
    for query_id, query_terms in queries.items():
        for query_term_id in tokenized_queries[query_id]:
            # TODO what happens when the query term doesn't occur in any document
            for doc_id, document_term_freq in inverted_index[query_term_id].items():
              
                #positional LM
                plm_models_wdi[query_term_id][doc_id] = score_fn(doc_id, query_term_id, document_term_freq)
                q_lm = query_lm[query_id][query_term_id]
                
                full_doc_length = len(index.document(doc_id)[1])
                for i in range(full_doc_length):
                    #for one word
                    
                    #final scores given query, doc and index
                    scores_qdi[query_id][doc_id][i] -= q_lm * np.log(q_lm / plm_models_wdi[query_term_id][doc_id][i]) 
                    print(i,scores_qdi[query_id][doc_id][i])
                current_score = score_sums[query_id][doc_id] #TODO check for zero and minus
                #scores_sum SHOULD BE RETURNED as the scores of each doc
                #print(max(list(scores_qdi[query_id][doc_id].values())))
                score_sums[query_id][doc_id] = max(current_score, max(list(scores_qdi[query_id][doc_id].values()))) 
                
    data = {}
    for query_id, document_scores in score_sums.items():
        doc_scores = []
        for doc_id in document_scores.keys():
            doc_scores.append((document_scores[doc_id], doc_id_to_ext_id[doc_id]))
        doc_scores.sort(key = lambda _: -_[0])
        data[query_id] = doc_scores
    
    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)

In [25]:
def PLM(int_document_id, query_term_id, document_term_freq, kernel = gaussian_kernel, sigma = 50, mu = 500):

    #please check, by ece
    
    #change kernel?
    #change mu according to optimized dirichlet hyperparameter!
    #sigma is given in the assignment
    
    #query_term_id = w
    #c_prime is for w,i
    
    full_doc_length = len(index.document(int_document_id)[1])
                
    print(full_doc_length)
    c_prime = np.zeros(full_doc_length)
    
    #get the positional word ids in the document, eliminate stop words where ind==0
    #TODO i and j after removing 0 or are we keeping the indices from the actual sequence?
    #WE ARE KEEPING EVERYTHING IN THE DOCUMENT
    
    indices = [ind for ind in index.document(int_document_id)[1]] 
    
    #denominator, virtual document lengths
    z = np.zeros(full_doc_length)
    
    for i in range(full_doc_length):
        
        #kernel outcomes for the given i over all j, they will be summed up for z[i]
        k = np.zeros(full_doc_length)
        
        for j in range(full_doc_length):
            
            cwj = 0.0
            if indices[j] == query_term_id:
                cwj = 1.0
            
            k[j] = kernel(i,j, sigma)
            c_prime[i] += cwj*k[j]
            
        z[i] = np.sum(k)   #sum over kernel(i,j) j=1 to N, not the final simplified version
        #NOT USING the sum of c_prime over all w, but using zi as the sum over kernels
        
    unigram_prob = collection_frequencies[query_term_id] / total_terms
    
    pm_wdi = (c_prime + mu * unigram_prob)/(z + mu)
    
    #language model per word given document and a position
    return pm_wdi

In [26]:
# combining the two functions above: 
#run_retrieval('tfidf', tfidf)
#run_retrieval('bm25', bm25)
#run_retrieval('jelinek_mercer', jelinek_mercer)
#run_retrieval('dirichlet_prior', dirichlet_prior)
#run_retrieval('absolute_discounting', absolute_discounting)
run_retrieval_plm('PLM', PLM)
# TODO implement the rest of the retrieval functions 

# TODO implement tools to help you with the analysis of the results.

Retrieving using PLM
874
0 -54819.981239
1 -54825.1889608
2 -54830.3843463
3 -54835.5653755
4 -54840.7300488
5 -54845.8763894
6 -54851.0024452
7 -54856.1062911
8 -54861.1860312
9 -54866.2398006
10 -54871.2657672
11 -54876.2621339
12 -54881.2271402
13 -54886.159064
14 -54891.056223
15 -54895.9169766
16 -54900.739727
17 -54905.5229208
18 -54910.2650505
19 -54914.9646552
20 -54919.6203224
21 -54924.2306882
22 -54928.7944392
23 -54933.3103126
24 -54937.7770974
25 -54942.193635
26 -54946.5588197
27 -54950.8715996
28 -54955.1309763
29 -54959.3360059
30 -54963.4857992
31 -54967.5795216
32 -54971.6163931
33 -54975.5956889
34 -54979.5167388
35 -54983.3789273
36 -54987.181693
37 -54990.9245291
38 -54994.606982
39 -54998.2286519
40 -55001.7891914
41 -55005.2883056
42 -55008.7257508
43 -55012.1013346
44 -55015.4149144
45 -55018.6663971
46 -55021.8557378
47 -55024.9829393
48 -55028.0480509
49 -55031.0511673
50 -55033.9924279
51 -55036.8720151
52 -55039.6901538
53 -55042.4471098
54 -55045.143189
55 

287 -55072.5031888
288 -55067.894259
289 -55062.9281517
290 -55057.5801303
291 -55051.8240436
292 -55045.6322766
293 -55038.9757043
294 -55031.8236476
295 -55024.1438334
296 -55015.9023583
297 -55007.0636584
298 -54997.5904837
299 -54987.4438811
300 -54976.583184
301 -54964.9660116
302 -54952.5482785
303 -54939.2842148
304 -54925.1263996
305 -54910.0258074
306 -54893.9318694
307 -54876.7925513
308 -54858.5544468
309 -54839.1628905
310 -54818.5620884
311 -54796.6952691
312 -54773.5048538
313 -54748.9326481
314 -54722.9200531
315 -54695.4082979
316 -54666.3386913
317 -54635.6528924
318 -54603.2931996
319 -54569.2028549
320 -54533.326363
321 -54495.6098216
322 -54456.001261
323 -54414.4509888
324 -54370.9119374
325 -54325.3400094
326 -54277.6944172
327 -54227.9380128
328 -54176.0376029
329 -54121.964246
330 -54065.6935252
331 -54007.2057954
332 -53946.4863989
333 -53883.5258464
334 -53818.3199619
335 -53750.8699868
336 -53681.1826437
337 -53609.2701569
338 -53535.1502306
339 -53458.845984

KeyboardInterrupt: 

In [11]:

class TrecAPI:
    def __init__(self, trec_path = "../../trec_eval/trec_eval"):
        self.trec_path = os.path.abspath(trec_path)
    
    def evaluate(self, test_file_name, prediction_file_name, metrics_to_capture=None, granular = False):
        """
        TODO
        """
        if metrics_to_capture is None:
            metrics_to_capture = {"ndcg_cut_10", "map_cut_1000", "P_5", "recall_1000"}
        try:
            command = [self.trec_path, "-m", "all_trec", "-q", test_file_name, prediction_file_name]
            output = subprocess.check_output(command, universal_newlines=True)
            data = collections.defaultdict(dict)
            for line in output.split("\n"):
                # ignore empty lines
                if line.strip() == "":
                    continue
                    
                metric, query, value = line.split("\t")
                
                if not granular and query != "all":
                    continue
                    
                
                metric = metric.strip()
                    
                # ignore metrics we don't care about
                if metric not in metrics_to_capture:
                    continue
                
                # relstring is a binary string, don't convert to float
                if metric not in {"relstring"}:
                    value = float(value)
                
                data[query][metric] = value 
            
            if not granular:
                return data["all"]
            
            return data
        except subprocess.CalledProcessError as e:
            print(e.output)
            return None
        

trec = TrecAPI()
results = trec.evaluate("./ap_88_89/qrel_validation", "./dirichlet_prior.run")
results

{'P_5': 0.2133,
 'map_cut_1000': 0.173,
 'ndcg_cut_10': 0.2382,
 'recall_1000': 0.5728}

In [20]:
def grid_eval(grid, score_fn, test_file, verbose = False, output_folder = "grid_results/"):
    """
    Evaluate a scoring method on a grid of possible values.
    grid : a dictionary of `param -> list of possible values`
    scoring_method: the scoring method to evaluate
    returns TODO
    """
    # convert to an list because indexing is required
    grid = list(grid.items())
    param_lengths = tuple([len(grid_item[1]) for grid_item in grid])
    print("Running grid eval on {} possible combinations".format(np.prod(param_lengths)))
    
    all_indices = np.ndindex(param_lengths)
    grid_results = []
    for index, values in enumerate(all_indices):
        param_grid = {}
        for param_index, param_val_index in enumerate(values):
            param_name, param_values = grid[param_index]
            param_grid[param_name] = param_values[param_val_index]
        if verbose:
            print("Running evaluation {} of {}, with params: {}".format(index + 1, np.prod(param_lengths), param_grid))
        output_path = os.path.join(output_folder, "{}_{}".format(score_fn.__name__, index))
        # construct the scoring function
        part_score_fn = functools.partial(score_fn, **param_grid) 
        run_retrieval(output_path, part_score_fn)
        metrics = trec.evaluate(test_file, output_path + ".run")
        grid_results.append((param_grid, metrics))
    return grid_results

    
grid_eval({"mu": [500, 1000, 5000]}, dirichlet_prior, "./ap_88_89/qrel_test", verbose=True)    

Running grid eval on 3 possible combinations
Running evaluation 1 of 3, with params: {'mu': 500}
Retrieving using grid_results/dirichlet_prior_0
Running evaluation 2 of 3, with params: {'mu': 1000}
Retrieving using grid_results/dirichlet_prior_1
Running evaluation 3 of 3, with params: {'mu': 5000}
Retrieving using grid_results/dirichlet_prior_2


[({'mu': 500},
  {'P_5': 0.2083,
   'map_cut_1000': 0.1023,
   'ndcg_cut_10': 0.1988,
   'recall_1000': 0.5245}),
 ({'mu': 1000},
  {'P_5': 0.2083,
   'map_cut_1000': 0.1017,
   'ndcg_cut_10': 0.197,
   'recall_1000': 0.5225}),
 ({'mu': 5000},
  {'P_5': 0.22,
   'map_cut_1000': 0.1061,
   'ndcg_cut_10': 0.2163,
   'recall_1000': 0.5118})]

In [12]:
def toss_coin(heads = 0.5):
    # head = True, Tails is False
    return True if random.random() < heads else False

In [15]:
def randomiser(list_a, list_b, N = 30):
    """
    TODO
    """
    assert len(list_a) == len(list_b)
    for i in range(N):
        new_a, new_b = [], []
        
        for index in range(len(list_a)):
            if toss_coin():
                # don't swap, copy from source
                new_a.append(list_a[index])
                new_b.append(list_b[index])
            else:
                new_a.append(list_b[index])
                new_b.append(list_a[index])
        mean_new_a = np.mean(new_a)
        mean_new_b = np.mean(new_b)
        
        yield mean_new_a, mean_new_b
        
a = np.transpose(list(randomiser([1,2,3],[4,5,6])))
print(a)
z, p = stats.ttest_rel(a[0],a[1])

print(z,p)

[[4. 4. 5. 4. 5. 4. 3. 4. 3. 4. 3. 3. 4. 4. 4. 4. 2. 4. 4. 3. 4. 2. 5. 4.
  5. 3. 3. 4. 3. 4.]
 [3. 3. 2. 3. 2. 3. 4. 3. 4. 3. 4. 4. 3. 3. 3. 3. 5. 3. 3. 4. 3. 5. 2. 3.
  2. 4. 4. 3. 4. 3.]]
1.6282258452024017 0.11429429746020744


In [16]:
def bonferroni(groups, alpha = 0.05):
    comparisons = math.factorial(groups)/(math.factorial(2)*math.factorial(groups - 2))
    return alpha/comparisons

### Task 2: Latent Semantic Models (LSMs) [15 points] ###

In this task you will experiment with applying distributional semantics methods ([LSI](http://lsa3.colorado.edu/papers/JASIS.lsi.90.pdf) **[5 points]** and [LDA](https://www.cs.princeton.edu/~blei/papers/BleiNgJordan2003.pdf) **[5 points]**) for retrieval.

You do not need to implement LSI or LDA on your own. Instead, you can use [gensim](http://radimrehurek.com/gensim/index.html). An example on how to integrate Pyndri with Gensim for word2vec can be found [here](https://github.com/cvangysel/pyndri/blob/master/examples/word2vec.py). For the remaining latent vector space models, you will need to implement connector classes (such as `IndriSentences`) by yourself.

In order to use a latent semantic model for retrieval, you need to:
   * build a representation of the query **q**,
   * build a representation of the document **d**,
   * calculate the similarity between **q** and **d** (e.g., cosine similarity, KL-divergence).
     
The exact implementation here depends on the latent semantic model you are using. 
   
Each of these LSMs come with various hyperparameters to tune. Make a choice on the parameters, and explicitly mention the reasons that led you to these decisions. You can use the validation set to optimize hyper parameters you see fit; motivate your decisions. In addition, mention clearly how the query/document representations were constructed for each LSM and explain your choices.

In this experiment, you will first obtain an initial top-1000 ranking for each query using TF-IDF in **Task 1**, and then re-rank the documents using the LSMs. Use TREC Eval to obtain the results and report on `NDCG@10`, Mean Average Precision (`MAP@1000`), `Precision@5` and `Recall@1000`.

Perform significance testing **[5 points]** (similar as in Task 1) in the class of semantic matching methods.

In [17]:
import gensim

In [18]:
def document_token_iterator():
    for int_doc_id in range(index.document_base(), index.maximum_document()):
        ext_doc_id, doc_token_ids = index.document(int_doc_id)
        ll = [id2token[token_id].lower() for token_id in doc_token_ids if token_id > 0]
        yield ll

In [19]:
def get_dictionary(dict_file_name = "./index.dict"):
    
    if os.path.exists(dict_file_name):
        print("Loading existing dictionary")
        return gensim.corpora.Dictionary.load(dict_file_name)
    print("Creating dictionary, and saving...")
    gensim_dict = gensim.corpora.Dictionary(document_token_iterator())
    gensim_dict.save(dict_file_name)
    
    return gensim_dict
gensim_dict = get_dictionary()

Loading existing dictionary


In [20]:
gensim_dict.token2id

{'geldof': 123610,
 'respicio': 20514,
 'baturisnky': 189286,
 'chinned': 189437,
 'vanderinden': 116858,
 'danzel': 191554,
 'brackman': 128475,
 'baybank': 151669,
 'owns': 1000,
 'tipich': 246691,
 'antitoxins': 221810,
 'taeju': 128393,
 'chope': 39253,
 'sowinski': 169866,
 'appeled': 129080,
 'lingington': 227885,
 'kl1': 223050,
 'giannasio': 87548,
 'trickling': 41706,
 'gijung': 218468,
 'vanhorssen': 151556,
 'greyhound': 28549,
 'bookfairs': 174200,
 'whynot': 189183,
 'croup': 85844,
 'pictorialism': 94376,
 'stanhope': 122729,
 'aerican': 186389,
 'korotich': 26468,
 'ch53': 139174,
 '0047': 165515,
 'chalsty': 121903,
 'exprience': 189384,
 'hornberg': 265437,
 'repressed': 39647,
 'tenley': 228513,
 'postdoctoral': 128941,
 'baggot': 162096,
 'cutover': 150265,
 'clayeux': 138306,
 'wimbiscus': 211239,
 'cuddihy': 77528,
 'coporations': 127094,
 'biehn': 165922,
 'demmer': 229524,
 'cups': 39582,
 'khusainov': 96641,
 'delegitimized': 76084,
 '2558': 212137,
 'karyab': 4

In [21]:
def get_corpus(gensim_dict, corpus_file="ir1.mm"):
    if os.path.exists(corpus_file):
        print("Loading existing corpus")
        return gensim.corpora.MmCorpus(corpus_file)
    print("Creating corpus, and saving....")
    gensim.corpora.MmCorpus.serialize(corpus_file, (gensim_dict.doc2bow(doc) for doc in document_token_iterator()))
    return gensim.corpora.MmCorpus(corpus_file)
gensim_corpus = get_corpus(gensim_dict)

Loading existing corpus


In [71]:
num_topics = 256
lsi_model = gensim.models.lsimodel.LsiModel(corpus=gensim_corpus, id2word=gensim_dict, num_topics=num_topics)

In [24]:
lsi_model = gensim.models.lsimodel.LsiModel.load("lsi.model")

In [23]:
lsi_model.save("lsi.model")

In [25]:
lsi_model.show_topics()

[(0,
  '0.294*"percent" + 0.200*"1" + 0.199*"new" + 0.179*"million" + 0.144*"two" + 0.141*"000" + 0.138*"people" + 0.135*"government" + 0.130*"president" + 0.126*"2"'),
 (1,
  '-0.630*"percent" + -0.232*"0" + -0.225*"1" + -0.140*"2" + 0.121*"people" + -0.119*"million" + 0.113*"president" + 0.105*"two" + -0.105*"3" + 0.103*"government"'),
 (2,
  '-0.852*"y" + -0.475*"n" + -0.119*"democrats" + -0.118*"republicans" + -0.060*"x" + -0.046*"republican" + -0.041*"democrat" + -0.029*"smith" + -0.029*"yes" + -0.026*"d"'),
 (3,
  '-0.519*"percent" + 0.337*"1" + 0.232*"million" + -0.218*"bush" + 0.183*"cents" + -0.164*"dukakis" + 0.137*"2" + -0.116*"0" + 0.112*"higher" + 0.110*"lower"'),
 (4,
  '0.749*"0" + -0.229*"dukakis" + -0.207*"bush" + 0.125*"dem" + 0.124*"brown" + 0.124*"fox" + 0.122*"dogs" + 0.122*"lazy" + 0.120*"quick" + 0.119*"jumped"')]

In [26]:
lsi_model[dictionary.doc2bow(['washington'])]

[(0, 0.029250609503025184),
 (1, 0.024777029219027746),
 (2, 0.0012075688884451118),
 (3, -0.004433514426330202),
 (4, 0.010989860955648176)]

In [64]:
from scipy.spatial.distance import cosine as cosine_distance
cosine_distance_vectorized = np.vectorize(cosine_distance)


array([0., 0.])

In [66]:
class LSMScorer:
    def __init__(self, corpus, dictionary, model):
        self.model = model
        self.corpus = corpus
        self.dict = dictionary
        
        self.document_vectors = np.zeros((num_documents, self.model.num_topics))
        self.document_index = []
        for idx, int_doc_id in enumerate(range(index.document_base(), index.maximum_document())):
            ext_doc_id, doc_token_ids = index.document(int_doc_id)
            document_tokens = [id2token[token_id].lower() for token_id in doc_token_ids if token_id > 0]
            self.document_index.append(ext_doc_id)
            self.document_vectors[idx] = self._get_vector(document_tokens)    
    
    def _get_vector(self, tokens):
        vector = np.zeros(self.model.num_topics)
        # TODO why isn't the size always = num_topics? It's super strange! 
        # TODO check back why this is happening. it's a serious problem
        for topic, value in self.model[self.dict.doc2bow(tokens)]:
            vector[topic] = value
        return vector
    
    def score(self, query_tokens):
        # TODO order = 1 same, -1 not same
        # TODO cehck others as well
        # TODO some values are 
        # TODO have to use JS divergence
        query_vector = self._get_vector(query_tokens)
        distances = np.apply_along_axis(lambda _: cosine_distance(query_vector, _), 1, self.document_vectors)
        #for ext_doc_id, document_vector in self.document_vectors.items():
        #    results.append((cosine_distance(document_vector, query_vector), ext_doc_id))
        results = list(zip(distances, self.document_index))
        results.sort(key=lambda _: -_[0])
        return results

lsi_scorer = LSMScorer(gensim_corpus, gensim_dict, lsi_model)

In [68]:
def run_retrieval_lsm(model_name, scorer):
    """
    Runs a retrieval method for all the queries and writes the TREC-friendly results in a file.
    
    :param model_name: the name of the model (a string)
    :param scorer: the LSMScorer 
    """
    
    run_out_path = '{}.run'.format(model_name)

    #if os.path.exists(run_out_path):
    #    return

    retrieval_start_time = time.time()

    print('Retrieving using', model_name)
    
    data = {}
    start_time = time.time()
    for index, (query_id, query_tokens) in enumerate(tokenized_queries.items()):
        query_tokens = [id2token[token_id].lower() for token_id in query_tokens if token_id > 0]
        data[query_id] = lsi_scorer.score(query_tokens)
        if (index + 1) % 10 == 0:
            print("\t", index + 1 , "done.", time.time() - start_time, "elapsed" )

    with open(run_out_path, 'w') as f_out:
        write_run(
            model_name=model_name,
            data=data,
            out_f=f_out,
            max_objects_per_query=1000)


In [69]:
run_retrieval_lsm("lsi", lsi_model)

Retrieving using lsi


/media/samarth/Data/workspaces/uva/ir1/lib/python3.5/site-packages/scipy/spatial/distance.py:644: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


	 10 done. 55.72089147567749 elapsed
	 20 done. 108.03322577476501 elapsed
	 30 done. 161.94573855400085 elapsed
	 40 done. 216.38156366348267 elapsed
	 50 done. 272.4707000255585 elapsed
	 60 done. 326.4106206893921 elapsed
	 70 done. 382.64772748947144 elapsed
	 80 done. 435.04863953590393 elapsed
	 90 done. 487.8543703556061 elapsed
	 100 done. 541.6416058540344 elapsed
	 110 done. 598.9066052436829 elapsed
	 120 done. 655.685209274292 elapsed
	 130 done. 708.9158449172974 elapsed
	 140 done. 760.3229117393494 elapsed
	 150 done. 811.6108584403992 elapsed


In [70]:
trec.evaluate("./ap_88_89/qrel_validation", "lsi.run")

{'P_5': 0.0, 'map_cut_1000': 0.0, 'ndcg_cut_10': 0.0, 'recall_1000': 0.0007}

### Task 3:  Word embeddings for ranking [20 points] (open-ended) ###

First create word embeddings on the corpus we provided using [word2vec](http://arxiv.org/abs/1411.2738) -- [gensim implementation](https://radimrehurek.com/gensim/models/word2vec.html). You should extract the indexed documents using pyndri and provide them to gensim for training a model (see example [here](https://github.com/nickvosk/pyndri/blob/master/examples/word2vec.py)).
   
This is an open-ended task. It is left up you to decide how you will combine word embeddings to derive query and document representations. Note that since we provide the implementation for training word2vec, you will be graded based on your creativity on combining word embeddings for building query and document representations.

Note: If you want to experiment with pre-trained word embeddings on a different corpus, you can use the word embeddings we provide alongside the assignment (./data/reduced_vectors_google.txt.tar.gz). These are the [google word2vec word embeddings](https://code.google.com/archive/p/word2vec/), reduced to only the words that appear in the document collection we use in this assignment.

### Task 4: Learning to rank (LTR) [15 points] (open-ended) ###

In this task you will get an introduction into learning to rank for information retrieval.

You can explore different ways for devising features for the model. Obviously, you can use the retrieval methods you implemented in Task 1, Task 2 and Task 3 as features. Think about other features you can use (e.g. query/document length). Creativity on devising new features and providing motivation for them will be taken into account when grading.

For every query, first create a document candidate set using the top-1000 documents using TF-IDF, and subsequently compute features given a query and a document. Note that the feature values of different retrieval methods are likely to be distributed differently.

You are adviced to start some pointwise learning to rank algorithm e.g. logistic regression, implemented in [scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html).
Train your LTR model using 10-fold cross validation on the test set. More advanced learning to rank algorithms will be appreciated when grading.

### Task 4: Write a report [15 points; instant FAIL if not provided] ###

The report should be a PDF file created using the [sigconf ACM template](https://www.acm.org/publications/proceedings-template) and will determine a significant part of your grade.

   * It should explain what you have implemented, motivate your experiments and detail what you expect to learn from them. **[10 points]**
   * Lastly, provide a convincing analysis of your results and conclude the report accordingly. **[10 points]**
      * Do all methods perform similarly on all queries? Why?
      * Is there a single retrieval model that outperforms all other retrieval models (i.e., silver bullet)?
      * ...

**Hand in the report and your self-contained implementation source files.** Only send us the files that matter, organized in a well-documented zip/tgz file with clear instructions on how to reproduce your results. That is, we want to be able to regenerate all your results with minimal effort. You can assume that the index and ground-truth information is present in the same file structure as the one we have provided.
